In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [29]:
data = pd.read_csv('./retail_train.csv')
data.sample(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
420865,2489,29238310799,169,9855342,2,2.99,362,-2.99,1454,25,0.0,0.0
1975456,2390,40211137932,560,9655101,2,1.00,298,-1.00,1944,81,0.0,0.0
816035,983,31032663880,271,968417,1,0.50,356,-0.09,2205,39,0.0,0.0
74218,195,27785697900,68,1082185,1,1.25,384,0.00,1532,10,0.0,0.0
772575,1741,30883598360,260,6391167,1,2.00,382,-0.49,1810,38,0.0,0.0
1002555,652,31735443293,318,844165,1,1.34,424,-0.25,1850,46,0.0,0.0
424136,2168,29238747262,169,1085604,1,1.00,306,-0.29,1718,25,0.0,0.0
1986441,1702,40248492158,563,1107538,1,2.85,443,0.00,1813,81,0.0,0.0
1553256,1694,33659872048,454,985893,1,1.67,446,-0.33,1753,66,0.0,0.0
694394,198,30627910891,241,989069,1,2.50,420,-0.49,1218,35,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [12]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()

In [14]:
result.columns = ['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [41]:
def weighted_random_recommendation(data, n=5):
   
    items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()
    items_weights['weight'] = items_weights.sales_value / items_weights.sales_value.sum()
    
    items = np.array(data.item_id.unique())
    recs = np.random.choice(items, size=n, replace=False,
                            p=np.array(items_weights.weight)
                           )
    
    
    return recs.tolist()

In [52]:
%%time

weighted_recs = weighted_random_recommendation(data_test, n=5)


result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(data_test, n=5))
result.head()

CPU times: total: 40 s
Wall time: 41 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[15660189, 1004250, 14077737, 16223083, 898496]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15595984, 964734, 1087851, 15928076, 984353]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8014645, 969846, 2062212, 17179873, 1007632]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[13876441, 905539, 1015450, 1050079, 828393]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1101960, 10455584, 1077066, 13876580, 5581464]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [9]:
result = pd.read_csv('./predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [10]:
# your_code

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code